In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))


In [2]:
import sys
import os
import numpy as np

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/xgboost/xgboost4j/jars/xgboost4j_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/xgboost4j-spark_2.12-1.3.3.jar,/home/xgboost/xgboost4j/jars/spark-arrow-datasource-standard-1.1.0-jar-with-dependencies.jar pyspark-shell'
os.environ['ARROW_LIBHDFS3_DIR'] = '/home/xgboost/miniconda3/lib'
os.environ['LD_LIBRARY_PATH'] = '/home/xgboost/miniconda3/lib'

In [3]:
SPARK_HOME='/home/xgboost/spark-3.0.0-bin-hadoop2.7'
import findspark
findspark.init(SPARK_HOME)

In [4]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time, sleep
import subprocess
import math

In [5]:
home = '/home/xgboost'

def killsar(clients):
    for l in clients:
        out=!ssh $l "ps aux | grep -w sar | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    #for l in clients:
    #    out=!ssh $l "ps aux | grep -w pidstat | grep -v grep | tr -s ' ' | cut -d' ' -f2"
    #    for x in out:
    #        !ssh $l "kill $x > /dev/null 2>&1"
    #for l in clients:
    #    out=!ssh $l "ps aux | grep -w perf | grep -v grep | tr -s ' ' | cut -d' ' -f2"
    #    for x in out:
    #        !ssh root@$l "kill $x > /dev/null 2>&1"
    #for l in clients:
    #    !ssh $l "emon -stop > /dev/null 2>&1"
            
def startmonitor(clients,appid,**kwargs):
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
    
    #for l in clients:
    #    !ssh root@$l date
    
    killsar(clients)
    
    if kwargs.get("collect_emon",False):
        with open(home+"/emon.list",'w+') as f:
            f.write(emon_events)
        for l in clients:
            !scp {home}/emon.list {l}:{home}/emon.list  > /dev/null 2>&1
    
    perfsyscalls=kwargs.get("collect_perf_syscall",None)
    
    for l in clients:
        !mkdir -p $prof/$l/
        !ssh $l mkdir -p $prof/$l/
        !ssh $l "sar -o {prof}/{l}/sar.bin -r -u -d -n DEV 1 >/dev/null 2>&1 &"
        if kwargs.get("collect_pid",False):
            !ssh $l "jps | grep CoarseGrainedExecutorBackend | head -n 1 | cut -d' ' -f 1 | xargs  -I % pidstat -h -t -p % 1  > {prof}/{l}/pidstat.out  2>/dev/null &"
        !ssh $l 'cat /proc/uptime  | cut -d" " -f 1 | xargs -I ^ date -d "- ^ seconds"  +%s.%N' > $prof/$l/uptime.txt
        if kwargs.get("collect_sched",False):
            !ssh $l 'perf trace -e "sched:sched_switch" -C 8-15 -o {prof}/{l}/sched.txt -T -- sleep 10000 >/dev/null 2>/dev/null &'
        if kwargs.get("collect_emon",False):
            !ssh $l "emon -i {home}/emon.list -f {prof}/{l}/emon.rst >/dev/null 2>&1 & "
        if perfsyscalls is not None:
            !ssh $l "perf stat -e 'syscalls:sys_exit_poll,syscalls:sys_exit_epoll_wait' -a -I 1000 -o {prof}/{l}/perfstat.txt  >/dev/null 2>&1 & "
    return prof

def stopmonitor(clients, sc, appid, **kwargs):
    %cd ~
    
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
     
    killsar(clients)
    
    for l in clients:
        !ssh $l "sar -f {prof}/{l}/sar.bin -r > {prof}/{l}/sar_mem.sar;sar -f {prof}/{l}/sar.bin -u > {prof}/{l}/sar_cpu.sar;sar -f {prof}/{l}/sar.bin -d > {prof}/{l}/sar_disk.sar;sar -f {prof}/{l}/sar.bin -n DEV > {prof}/{l}/sar_nic.sar;" 
        !ssh $l "grep -rI xgbtck --no-filename /home/ubuntu/hadoop-2.7.0/logs/userlogs/{appid}/* | sed 's/^ //g'  > {prof}/{l}/xgbtck.txt"
        !scp -r $l:$prof/$l $prof/ > /dev/null 2>&1
        !ssh $l "jps | grep CoarseGrainedExecutorBackend | head -n 2 | tail -n 1 | cut -d' ' -f 1  | xargs -I % ps -To tid p %" > $prof/$l/sched_threads.txt
        #!ssh $l "emon -v " > $prof/$l/emonv.txt
        #!test -f $prof/$l/perfstat.txt && head -n 1 $prof/$l/perfstat.txt > $prof/$l/perfstarttime
    if sc is not None:
        sc.stop()

In [6]:
clients = ['sr572']
nrepeat = 1
executors_per_node = 1
nodes=len(clients)
cores_per_executor=8
task_per_core=1

cache_size=30000
arrow_datasource_jar = '/home/xgboost/xgboost4j/jars/spark-arrow-datasource-standard-1.1.0-jar-with-dependencies.jar'

def start_cntx():
    conf = SparkConf()\
        .set('spark.default.parallelism', '{:d}'.format(nodes*executors_per_node*cores_per_executor))\
        .set('spark.executor.instances', '{:d}'.format(executors_per_node*nodes))\
        .set('spark.rdd.compress', 'False')\
        .set('spark.files.maxPartitionBytes', '512m')\
        .set('spark.executor.cores','{:d}'.format(cores_per_executor))\
        .set('spark.executor.memory', '40g') \
        .set('spark.executor.memoryOverhead', '4g') \
        .set('spark.task.cpus', '{:d}'.format(task_per_core))\
        .set('spark.driver.memory', '10g')\
        .set('spark.executor.extraJavaOptions',\
            '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails')\
        .set('spark.driver.maxResultSize', '0')\
        .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')\
        .set('spark.memory.offHeap.enabled','True')\
        .set('spark.memory.offHeap.size','10g')\
        .set('spark.executorEnv.ARROW_LIBHDFS3_DIR', '/home/xgboost/miniconda3/lib')\
        .set('spark.executorEnv.LD_LIBRARY_PATH', '/home/xgboost/miniconda3/lib')\
        .set('spark.driver.extraClassPath', arrow_datasource_jar) \
        .set('spark.executor.extraClassPath', arrow_datasource_jar) \
        .set('xgboost.spark.arrow.optimization.enabled', 'True') \
        .setAppName('mortgage')

    spark = SparkSession.builder\
                .master('spark://sr572:7077')\
                .config(conf=conf)\
                .getOrCreate()
        
    sc = spark.sparkContext
    sc.setLogLevel('INFO')
    sc.addPyFile('/home/xgboost/xgboost4j/sparkxgb_1.24.zip')
    return sc, spark    

In [7]:
def train(numWorkers, label, features):
    from sparkxgb import XGBoostClassifier
    params = { 
        'labelCol': label,
        'featuresCols': features,
        'eta': 0.1,
        'gamma': 0.1,
        'missing': 0.0,
        'treeMethod': 'hist',
        'maxDepth': 8, 
        'maxLeaves': 256,
        'alpha':0.9,
        'objective':'reg:squarederror',
        'growPolicy': 'depthwise',
        'minChildWeight': 30.0,
        'reg_lambda': 1.0,
        'scalePosWeight': 2.0,
        'subsample': 1.0,
        'numRound': 100,
        'maxBin': 256,
        'nthread': cores_per_executor,
        'numWorkers': numWorkers,
        'singlePrecisionHistogram': True,
        'verbosity': 3     
    }
        
    #classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
    classifier = XGBoostClassifier(**params)
    return classifier

In [8]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    display(HTML("{} takes <font size=6pt color=red>{} seconds </font>".format(phrase, round(end - start, 2))))
    return result


def train_data_fn(nworker, train_data, label, features):
    classifier=train(nworker, label, features)
    return with_benchmark('Training', lambda: classifier.fit(train_data))

In [9]:
def load_parquet(path):
    label = 'delinquency_12'
    train_data = spark.read.format('arrow').load(path)
    # print("DataFrame schma: ", train_data.schema)
    features = [x for x in train_data.columns]
    print(f'features: ${features}')
    #train_data = train_data.coalesce(executors_per_node*nodes)   
    return (label,features,train_data)

In [10]:
SPARK_HOME='/home/xgboost/spark-3.0.0-bin-hadoop2.7'
!$SPARK_HOME/sbin/start-master.sh
!$SPARK_HOME/sbin/start-slave.sh spark://sr572:7077 -c 8

/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
starting org.apache.spark.deploy.master.Master, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.master.Master-1-sr572.out
/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
starting org.apache.spark.deploy.worker.Worker, logging to /home/xgboost/spark-3.0.0-bin-hadoop2.7/logs/spark-xgboost-org.apache.spark.deploy.worker.Worker-1-sr572.out


In [11]:
for i in range(nrepeat):
    sc, spark = start_cntx()
    appid = sc.applicationId
        
    label,features,train_data = load_parquet('file:///home/xgboost/data/xgboost_4M_float.dataframe.parquet')

    #startmonitor(clients, appid)
    model = train_data_fn(executors_per_node*nodes, train_data, label, features)
    model.nativeBooster.saveModel('/home/xgboost/models/withFeaturesCols.bin')
    #stopmonitor(clients, sc, appid)
    
    sc.stop()
    
    #acc_prof = '/home/xgboost/profile/'+appid      
    #xgbtck = acc_prof + '/*/xgbtck.txt'
    #dmat_create_times = !grep dmat_time --no-filename $xgbtck | cut -d' ' -f3
    #train_loop_times = !grep train_time --no-filename $xgbtck | cut -d' ' -f3
    #dmat_create_time = np.array([float(t) for t in dmat_create_times]).max()
    #train_loop_time = np.array([float(t) for t in train_loop_times]).max()
    #display(HTML(('Time spent in building DMatrix (sec): <font size=6pt color=red>{:f}</font>'.format(dmat_create_time))))
    #display(HTML(('Time spent in train loops (sec): <font size=6pt color=red>{:f}</font>'.format(train_loop_time))))

    sleep(10)

features: $['interest_rate', 'current_actual_upb', 'loan_age', 'remaining_months_to_legal_maturity', 'adj_remaining_months_to_maturity', 'msa', 'current_loan_delinquency_status', 'foreclosure_costs', 'prop_preservation_and_repair_costs', 'asset_recovery_costs', 'misc_holding_expenses', 'holding_taxes', 'net_sale_proceeds', 'credit_enhancement_proceeds', 'repurchase_make_whole_proceeds', 'other_foreclosure_proceeds', 'non_interest_bearing_upb', 'principal_forgiveness_upb', 'foreclosure_principal_write_off_amount', 'servicer_idx', 'mod_flag_idx', 'zero_balance_code_idx', 'repurchase_make_whole_proceeds_flag_idx', 'servicing_activity_indicator_idx', 'delinquency_12', 'orig_interest_rate', 'orig_upb', 'orig_loan_term', 'orig_ltv', 'orig_cltv', 'num_borrowers', 'dti', 'borrower_credit_score', 'num_units', 'zip', 'mortgage_insurance_percent', 'coborrow_credit_score', 'mortgage_insurance_type', 'datatime', 'orig_channel_idx', 'first_home_buyer_idx', 'loan_purpose_idx', 'property_type_idx', 'o

In [12]:
sc.stop()

In [13]:
!$SPARK_HOME/sbin/stop-slave.sh
!$SPARK_HOME/sbin/stop-master.sh

/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
stopping org.apache.spark.deploy.worker.Worker
/bin/bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
bash: /home/xgboost/miniconda3/lib/libtinfo.so.6: no version information available (required by bash)
stopping org.apache.spark.deploy.master.Master
